In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from psyke.utils.dataframe import get_discrete_features_supervised, get_discrete_dataset
from psyke.utils.logic import pretty_theory
from psyke import Extractor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
x, y = load_iris(return_X_y=True, as_frame=True)
x.columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
y = pd.DataFrame(y).replace({"target": {0: 'setosa', 1: 'virginica', 2: 'versicolor'}})
y.columns = ['iris']

dataset = x.join(y)

iris_features = get_discrete_features_supervised(dataset)
x = get_discrete_dataset(x, iris_features)

discretized = x.join(y)

In [3]:
_, original_test = train_test_split(dataset, test_size=0.5, random_state=1)
train, test = train_test_split(discretized, test_size=0.5, random_state=1)

predictor = KNeighborsClassifier(n_neighbors=7)
predictor.fit(train.iloc[:, :-1], train.iloc[:, -1])

extractor = Extractor.real(predictor, iris_features)
theory = extractor.extract(train)
print('Extracted rules:\n\n' + pretty_theory(theory))

Extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    PetalWidth =< 0.6474, SepalLength =< 5.3892, PetalLength not_in [2.2685, 4.865].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    SepalLength in [5.3892, 6.262], SepalWidth > 3.201, PetalLength not_in [2.2685, 4.865], PetalLength =< 4.865, PetalWidth not_in [0.6474, 1.6376], PetalWidth =< 1.6376.
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalLength > 4.865, SepalLength > 6.262, PetalWidth > 0.6474, SepalWidth > 2.872.
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalWidth > 1.6376, PetalLength > 2.2685, PetalLength not_in [2.2685, 4.865].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    SepalWidth =< 2.872, PetalLength > 2.2685, PetalLength =< 4.865, PetalWidth > 0.6474, PetalWidth =< 1.6376.
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    PetalLength > 2.2685, PetalLe

In [6]:
from psyke.utils.ontology import dataframe_to_ontology, rules_from_theory, individuals_from_dataframe

onto = dataframe_to_ontology(dataset)
individuals = individuals_from_dataframe(onto, original_test.iloc[:, :-1])
onto = rules_from_theory(onto, theory, dataset)

print(list(onto.classes()), list(onto.individuals()), list(onto.rules()))

[iris.Iris] [iris.iris1, iris.iris2, iris.iris3, iris.iris4, iris.iris5, iris.iris6, iris.iris7, iris.iris8, iris.iris9, iris.iris10, iris.iris11, iris.iris12, iris.iris13, iris.iris14, iris.iris15, iris.iris16, iris.iris17, iris.iris18, iris.iris19, iris.iris20, iris.iris21, iris.iris22, iris.iris23, iris.iris24, iris.iris25, iris.iris26, iris.iris27, iris.iris28, iris.iris29, iris.iris30, iris.iris31, iris.iris32, iris.iris33, iris.iris34, iris.iris35, iris.iris36, iris.iris37, iris.iris38, iris.iris39, iris.iris40, iris.iris41, iris.iris42, iris.iris43, iris.iris44, iris.iris45, iris.iris46, iris.iris47, iris.iris48, iris.iris49, iris.iris50, iris.iris51, iris.iris52, iris.iris53, iris.iris54, iris.iris55, iris.iris56, iris.iris57, iris.iris58, iris.iris59, iris.iris60, iris.iris61, iris.iris62, iris.iris63, iris.iris64, iris.iris65, iris.iris66, iris.iris67, iris.iris68, iris.iris69, iris.iris70, iris.iris71, iris.iris72, iris.iris73, iris.iris74, iris.iris75, iris.iris76, iris.iri

In [10]:
from owlready2 import *
owlready2.JAVA_EXE = "C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath\\java.exe"

sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True, debug = 0)
for individual, output in zip(individuals, original_test.iloc[:, -1]):
    print(individual.iris, output)

setosa setosa
virginica virginica
virginica virginica
setosa setosa
versicolor versicolor
None virginica
versicolor versicolor
setosa setosa
setosa setosa
versicolor versicolor
virginica virginica
setosa setosa
versicolor versicolor
virginica virginica
virginica virginica
setosa setosa
virginica virginica
virginica virginica
setosa setosa
setosa setosa
virginica virginica
virginica virginica
versicolor virginica
setosa setosa
versicolor versicolor
virginica virginica
setosa setosa
setosa setosa
virginica virginica
versicolor versicolor
virginica virginica
versicolor versicolor
virginica virginica
versicolor versicolor
versicolor versicolor
setosa setosa
virginica virginica
setosa setosa
virginica virginica
versicolor versicolor
versicolor versicolor
setosa setosa
virginica versicolor
versicolor versicolor
virginica virginica
versicolor versicolor
setosa setosa
setosa setosa
setosa setosa
virginica virginica
setosa setosa
setosa setosa
versicolor versicolor
versicolor versicolor
versico

In [ ]:
from pandas.core.dtypes.common import is_numeric_dtype, is_string_dtype
from owlready2 import *
import owlready2
owlready2.JAVA_EXE = "C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath\\java.exe"

onto = get_ontology("iris.rdf")

with onto:
    class Iris(Thing): pass
    for column in dataset.columns:
        if is_numeric_dtype(dataset[column]):
            type(column, (Iris >> float, FunctionalProperty), {})
        elif is_string_dtype(dataset[column]):
            type(column, (Iris >> str, FunctionalProperty), {})
        else:
            raise TypeError

    # for _, row in dataset.iterrows():
    #    string_row = "Iris("
    #    for name, value in zip(row.index, row):
    #        if isinstance(value, numbers.Number):
    #            string_row += f"{name}={value},"
    #        elif isinstance(value, str):
    #            string_row += f"{name}='{value}',"
    #        else:
    #            raise TypeError
    #    eval(string_row[:-1] + ")")

    rule5 = Imp()
    #rule2 = Imp()
    rule5.set_as_rule("""Iris(?i), PetalLength(?i, ?pl), SepalLength(?i, ?sl), PetalWidth(?i, ?pw), SepalWidth(?i, ?sw), greaterThanOrEqual(?pl, 5) -> iris(?i, 'setosa')""")
    #rule2.set_as_rule("""Iris(?i), PetalLength(?i, ?pl), SepalLength(?i, ?sl), PetalWidth(?i, ?pw), SepalWidth(?i, ?sw), lessThan(?pl, 5) -> iris(?i, 'versicolor')""")

print(list(onto.classes()), list(onto.individuals()), list(onto.rules()))

# for prop in onto.iris1.get_properties():
#    for value in prop[onto.iris1]:
#        print(".%s == %s" % (prop.python_name, value))
    #class Iris(Thing): pass
    #class petalLength(Iris >> float, FunctionalProperty): pass
    #class petalWidth(Iris >> float, FunctionalProperty): pass
    #class sepalLength(Iris >> float, FunctionalProperty): pass
    #class sepalWidth(Iris >> float, FunctionalProperty): pass
    #class irisClass(Iris >> str, FunctionalProperty): pass
#onto.save(file = "file.rdf", format = "rdfxml")
sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
#iris15 = eval("Iris(PetalLength = 2.5, PetalWidth = 3.6, SepalLength = 6.3, SepalWidth = 1.3)")
#iris2 = Iris(PetalLength = 12.5, PetalWidth = 3.6, SepalLength = 6.3, SepalWidth = 1.3)

#print(iris15.iris, iris2.iris)
#onto.destroy()

In [ ]:
with onto:
    print(list(default_world.inconsistent_classes()))

In [ ]:
onto.iris1.get_properties()

In [ ]:
onto.iris1.SepalWidth